## Engineering Assignment 2: Bridge Load Analysis
**Brian Capellan-Santos** 

## Introduction

This report analyzes bridges subjected to multiple loads, simulating real-world vehicle crossings.

Using the superposition principle bending moments from multiple axles are summed to determine:
- Maximum bending moments
- Critical vehicle positions
- Comparison to simplified single-load models
- Load amplification factors

The analysis ensures safe bridge design under realistic traffic conditions.

## calculate_single_moment

This function calculates the internal bending moment at a specific location on a simply supported bridge under a single point load.

In [1]:
def calculate_single_moment(bridge_length, load, load_position, x):    
    b = bridge_length - load_position
    
    if x <= load_position:
        return (load * b * x) / bridge_length
    else:
        return (load * load_position * (bridge_length - x)) / bridge_length


## Function 1: calculate_multiple_moments

This function calculates the total bending moment at a given location on a bridge due to multiple loads using superposition. Each load's contribution is calculated with the single-load formula.

In [2]:
def calculate_multiple_moments(bridge_length, loads, positions, x):    
    total_moment = 0
    for load, pos in zip(loads, positions):
        # Only include loads on the bridge
        if 0 <= pos <= bridge_length:
            total_moment += calculate_single_moment(bridge_length, load, pos, x)
    
    return total_moment

## Function 2: find_max_moment_multiple

This function finds the maximum bending moment along the bridge under multiple loads and the position where it occurs.

In [3]:
def find_max_moment_multiple(bridge_length, loads, positions):   
    # Generate test points and include load positions on the bridge
    test_points = list(np.linspace(0, bridge_length, 101))
    valid_positions = [p for p in positions if 0 <= p <= bridge_length]
    combined_points = sorted(set(test_points + valid_positions))
    
    # Calculate moments
    moments = [calculate_multiple_moments(bridge_length, loads, positions, x) for x in combined_points]
    
    max_moment = max(moments)
    max_location = combined_points[np.argmax(moments)]
    
    return max_moment, max_location

## Function 3: create_vehicle_loads

Generates axle loads and positions for a vehicle type given the front axle location.

In [4]:
def create_vehicle_loads(vehicle_type, front_position):
    """
    Returns loads and positions for vehicle axles.
    
    Vehicle Types:
    - car: [6, 9] kN, 2.5m spacing
    - bus: [45, 75] kN, 6m spacing
    - truck: [50, 70, 70] kN, 3.5m & 1.4m spacing
    
    Returns:
    Tuple: (loads_list, positions_list)
    """
    if vehicle_type == 'car':
        loads = [6, 9]
        positions = [front_position, front_position - 2.5]
    elif vehicle_type == 'bus':
        loads = [45, 75]
        positions = [front_position, front_position - 6.0]
    elif vehicle_type == 'truck':
        loads = [50, 70, 70]
        positions = [front_position, front_position - 3.5, front_position - (3.5 + 1.4)]
    else:
        raise ValueError("Vehicle type must be 'car', 'bus', or 'truck'")
    
    return loads, positions


## Function 4: analyze_vehicle_crossing

Simulates a vehicle crossing the bridge to find the **critical front axle position** that causes the **maximum bending moment**.

In [5]:
import numpy as np
def analyze_vehicle_crossing(bridge_length, vehicle_type, num_positions=50):
    start = -10
    end = bridge_length + 10
    front_positions = np.linspace(start, end, num_positions)
    
    max_moment = 0
    critical_position = None
    max_location = None
    all_maxima = []
    
    for front in front_positions:
        loads, positions = create_vehicle_loads(vehicle_type, front)
        moment, location = find_max_moment_multiple(bridge_length, loads, positions)
        all_maxima.append(moment)
        if moment > max_moment:
            max_moment = moment
            critical_position = front
            max_location = location
    
    return {
        'critical_front_position': critical_position,
        'max_moment': max_moment,
        'max_moment_location': max_location,
        'all_maxima': all_maxima
    }


## Scenario 1: City Bus on 25m Urban Bridge

In [8]:
truck_result = analyze_vehicle_crossing(30, 'truck', num_positions=50)
truck_result

{'critical_front_position': 18.571428571428573,
 'max_moment': 1288.2843537414967,
 'max_moment_location': 15.071428571428573,
 'all_maxima': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  10.134666111342485,
  58.72553102873805,
  103.84561988060537,
  145.49493266694435,
  230.81632653061234,
  345.84867416354285,
  484.54921560460923,
  610.060807996668,
  722.3834513397197,
  821.5171456337637,
  907.4618908788004,
  989.9510204081633,
  1072.1369151742329,
  1141.1338608912952,
  1196.9418575593504,
  1239.560905178398,
  1268.9910037484383,
  1285.232153269471,
  1288.2843537414967,
  1278.1476051645147,
  1254.8219075385257,
  1218.3072608635289,
  1174.020807996668,
  1120.1758822712757,
  1053.1420074968764,
  972.9191836734692,
  879.5074108010547,
  772.9066888796336,
  653.1170179092044,
  520.1383978897676,
  409.23113702623914,
  297.07370262390646,
  175.19809523809462,
  69.13215743440227,
  0.142847424683928,
  0,
  0,
  0,
  0,
  0]}

In [10]:
## Scenario 2: Delivery Truck on 30m Highway Bridge

In [11]:
truck_result = analyze_vehicle_crossing(30, 'truck', num_positions=50)
truck_result

{'critical_front_position': 18.571428571428573,
 'max_moment': 1288.2843537414967,
 'max_moment_location': 15.071428571428573,
 'all_maxima': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  10.134666111342485,
  58.72553102873805,
  103.84561988060537,
  145.49493266694435,
  230.81632653061234,
  345.84867416354285,
  484.54921560460923,
  610.060807996668,
  722.3834513397197,
  821.5171456337637,
  907.4618908788004,
  989.9510204081633,
  1072.1369151742329,
  1141.1338608912952,
  1196.9418575593504,
  1239.560905178398,
  1268.9910037484383,
  1285.232153269471,
  1288.2843537414967,
  1278.1476051645147,
  1254.8219075385257,
  1218.3072608635289,
  1174.020807996668,
  1120.1758822712757,
  1053.1420074968764,
  972.9191836734692,
  879.5074108010547,
  772.9066888796336,
  653.1170179092044,
  520.1383978897676,
  409.23113702623914,
  297.07370262390646,
  175.19809523809462,
  69.13215743440227,
  0.142847424683928,
  0,
  0,
  0,
  0,
  0]}

## Scenario 3: Truck + Car Convoy on 30m Bridge

In [12]:
# Helper: combine truck + car
def create_convoy_load(truck_front, bridge_length):
    truck_loads, truck_positions = create_vehicle_loads('truck', truck_front)
    car_front = truck_front - 10
    car_loads, car_positions = create_vehicle_loads('car', car_front)
    loads = truck_loads + car_loads
    positions = truck_positions + car_positions
    return loads, positions

def analyze_convoy_crossing(bridge_length, num_positions=50):
    start = -10
    end = bridge_length + 15
    front_positions = np.linspace(start, end, num_positions)
    
    max_moment = 0
    critical_position = None
    max_location = None
    all_maxima = []
    
    for truck_front in front_positions:
        loads, positions = create_convoy_load(truck_front, bridge_length)
        moment, location = find_max_moment_multiple(bridge_length, loads, positions)
        all_maxima.append(moment)
        if moment > max_moment:
            max_moment = moment
            critical_position = truck_front
            max_location = location
    
    return {
        'critical_front_position': critical_position,
        'max_moment': max_moment,
        'max_moment_location': max_location,
        'all_maxima': all_maxima
    }

convoy_result = analyze_convoy_crossing(30, num_positions=50)
convoy_result


{'critical_front_position': 18.061224489795915,
 'max_moment': 1339.0555081216162,
 'max_moment_location': 14.561224489795915,
 'all_maxima': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5.084686935998884,
  58.72553102873796,
  108.16673608218795,
  153.40830209634876,
  259.1805497709286,
  402.91156462585025,
  548.9536304317644,
  679.0370678883797,
  793.1618769956962,
  892.1360544217687,
  986.889795918367,
  1082.4476190476187,
  1167.948469387755,
  1236.5530091628486,
  1287.939028876857,
  1322.106528529779,
  1339.0555081216162,
  1338.785967652367,
  1321.2979071220325,
  1288.1013265306121,
  1246.914321116202,
  1188.5087956407056,
  1112.8847501041237,
  1020.0421845064559,
  909.9810988477026,
  782.7014931278635,
  638.2033673469389,
  493.83921352214344,
  365.48679092044966,
  241.23865056226632,
  137.99411703456917,
  74.04277731500788,
  64.44111828404834,
  53.686224489795954,
  41.67143898375681,
  28.396761765930897,
  16.720376926280736,
  7.580435235318632,

## Question 1

In [13]:
# Compare bus at centroid, front at quarter, rear at quarter
bus_positions = {'centroid': 12.5, 'front_quarter': 6.25, 'rear_quarter': 6.25+6.0}
bus_moments = {}
for pos_name, front_pos in bus_positions.items():
    loads, positions = create_vehicle_loads('bus', front_pos)
    moment, location = find_max_moment_multiple(25, loads, positions)
    bus_moments[pos_name] = moment

bus_moments


{'centroid': 525.0, 'front_quarter': 225.0, 'rear_quarter': 520.2}

## Question 2

The maximum moment may not coincide with the front axle; it often occurs under the heavier rear axle due to lever arm effects. This demonstrates why bridges must be analyzed at multiple positions.

## Question 3

In [15]:
percent_increase = ((convoy_result['max_moment'] - truck_result['max_moment']) / truck_result['max_moment']) * 100
percent_increase


3.9409897537502077

## Question 4

In [16]:
# Bus centroid calculation
P = [45, 75]
d = [0, 6.0] # distances from front axle
cg = sum([Pi*di for Pi, di in zip(P, d)]) / sum(P)
cg

3.75

In [17]:
# Single load at centroid
total_load = sum(P)
single_load_moment = calculate_single_moment(25, total_load, 12.5-cg, 12.5) # approx midspan
single_load_moment

525.0

Compare to multi-axle max: percentage difference = (multi-axle - single load)/multi-axle * 100%

## Question 5

In [19]:
simplified_single_load = 190
single_moment = calculate_single_moment(30, simplified_single_load, 15, 15)
actual_max = truck_result['max_moment']
percent_error = abs(single_moment - actual_max) / actual_max * 100
percent_error


10.612225931444964

The simplification may be unconservative if the simplified single load underestimates maximum bending moment.